In [2]:
import ollama

modelfile = {
    "temperature": 0.7,
    #"template": "{{ if .System }}<|start_header_id|>system<|end_header_id|>\n\n{{ .System }}<|eot_id|>{{ end }}{{ if .Prompt }}<|start_header_id|>user<|end_header_id|>\n\n{{ .Prompt }}<|eot_id|>{{ end }}<|start_header_id|>Alice<|end_header_id|>\n\n{{ .Response }}<|eot_id|>",
    }

response = ollama.chat(
                    model="qwen2.5-coder:7b-instruct-q4_K_M",
                    messages=[
                        {
                            "role": "system", 
                            "content": ""
                        },
                        {
                            "role": "user",
                            "content": "\n[Action Options]\n1) MOVE <target>, <target> can only be a cube.\n2) SWEEP <target>, this moves the groom so it pushes the <target> into dustpan, only Bob can SWEEP, Alice must WAIT in front of the same <target> cube when Bob SWEEP.\n3) WAIT, stays at the current spot.\n4) DUMP, only when there are one or more cubes in the dustpan, Alice can DUMP it into trash_bin.\nOnly SWEEP a cube after both robots MOVEed to the cube.\n[Action Output Instruction]\nMust first output 'EXECUTE\n', then give exactly one action per robot, put each on a new line.\nExample#1: 'EXECUTE\nNAME Alice ACTION MOVE red_cube\nNAME Bob ACTION MOVE red_cube\n'\nExample#2: 'EXECUTE\nNAME Alice ACTION WAIT\nNAME Bob ACTION SWEEP red_cube\n'\nExample#3: 'EXECUTE\nNAME Alice ACTION DUMP\nNAME Bob ACTION MOVE green_cube\n'\n\n\n\nYou are a robot called Alice, and you are collaborating with Bob to sweep up all the cubes on the table.\nYou hold a dustpan. \nTo sweep up a cube, you and Bob must get close to it by MOVE to opposite sides of the same cube. You must WAIT at the same cube while Bob SWEEPs.\nTalk with Bob to coordinate together and decide which cube to sweep up first.\nAt the current round:\nYour gripper is at (0.3, -0.1, 0.5), holding dustpan, in front of red_cube with distance: 0.80, in front of green_cube with distance: 0.62, in front of blue_cube with distance: 0.43\nred_cube is at (1.0, 0.5, 0.2), on the table; \ngreen_cube is at (0.7, 0.5, 0.2), on the table; \nblue_cube is at (0.3, 0.5, 0.2), on the table; \nThink step-by-step about the task and Bob's response. Carefully check and correct them if they made a mistake. \nImprove your plans if given [Environment Feedback].\nNever forget you are Alice!\nRespond very concisely but informatively, and do not repeat what others have said. Discuss with others to come up with the best plan.\nPropose exactly one action for yourself at the **current** round, select from [Action Options].\nEnd your response by either: 1) output PROCEED, if the plans require further discussion; 2) If everyone has made proposals and got approved, output the final plan, must strictly follow [Action Output Instruction]!\n\n\n" + "You are Alice, your response is:"
                        }
                    ],
                    options=modelfile,
                    stream=False
                )
print(response)

{'model': 'qwen2.5-coder:7b-instruct-q4_K_M', 'created_at': '2024-11-02T14:05:57.206840608Z', 'message': {'role': 'assistant', 'content': 'EXECUTE\nNAME Alice ACTION MOVE green_cube\nPROCEED'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3144685619, 'load_duration': 15047256, 'prompt_eval_count': 587, 'prompt_eval_duration': 1213435000, 'eval_count': 13, 'eval_duration': 1779226000}


In [3]:
response["message"]["content"]

'EXECUTE\nNAME Alice ACTION PICK apple PATH [(0.29, 0.06, 0.8), (0.2, -0.05, 0.7), (-0.1, -0.05, 0.65), (-0.73, 0.40, 0.5)]\nNAME Alice ACTION PLACE apple bin_front_left PATH [(0.25, 0.41, 0.4), (0.3, 0.42, 0.45), (0.2, 0.39, 0.46), (0.25, 0.41, 0.36)]'